# Remote Plugin Proxy

> Bridge between Host application and isolated Worker processes

In [ ]:
#| default_exp core.proxy

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import json
import os
import socket
import subprocess
import time
from pathlib import Path
from typing import Any, AsyncGenerator, Dict, Generator, Optional

import httpx

from cjm_plugin_system.core.interface import FileBackedDTO, PluginInterface

The `RemotePluginProxy` implements `PluginInterface` but forwards all calls over HTTP to a Worker process running in an isolated environment.

```
Host Application                        Isolated Environment
┌────────────────────┐                 ┌─────────────────────┐
│  PluginManager     │                 │  Conda Env (GPU)    │
│    │               │                 │    │                │
│    ▼               │     HTTP        │    ▼                │
│  RemotePluginProxy │◄───────────────▶│  Universal Worker   │
│  (implements       │   localhost     │    │                │
│   PluginInterface) │                 │    ▼                │
│                    │                 │  Actual Plugin      │
└────────────────────┘                 └─────────────────────┘
```

Key responsibilities:

1. **Process Management**: Launch worker subprocess with correct Python interpreter
2. **Port Allocation**: Find free port, pass to worker via CLI
3. **Zero-Copy Transfer**: Detect `FileBackedDTO` objects and serialize to temp files
4. **Dual Interface**: Sync methods for scripts, async methods for FastHTML

## RemotePluginProxy

In [ ]:
#| export
class RemotePluginProxy(PluginInterface):
    """Proxy that forwards plugin calls to an isolated Worker subprocess."""
    
    def __init__(
        self,
        manifest: Dict[str, Any] # Plugin manifest with python_path, module, class, etc.
    ):
        """Initialize proxy and start the worker process."""
        self.manifest = manifest
        self.process: Optional[subprocess.Popen] = None
        self.port = self._get_free_port()
        self.base_url = f"http://127.0.0.1:{self.port}"
        self._start_process()

    @property
    def name(self) -> str: # Plugin name from manifest
        """Plugin name."""
        return self.manifest.get('name', 'unknown')
    
    @property
    def version(self) -> str: # Plugin version from manifest
        """Plugin version."""
        return self.manifest.get('version', '0.0.0')

    def _get_free_port(self) -> int:
        """Find an available port for the worker."""
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.bind(('', 0))
            return s.getsockname()[1]

    def _start_process(self) -> None:
        """Launch the worker subprocess."""
        python_path = self.manifest['python_path']
        cmd = [
            python_path,
            "-m", "cjm_plugin_system.core.worker",
            "--module", self.manifest['module'],
            "--class", self.manifest['class'],
            "--port", str(self.port),
            "--ppid", str(os.getpid())  # Enable suicide pact
        ]
        
        # Merge environment variables from manifest
        env = dict(os.environ)
        env.update(self.manifest.get('env_vars', {}))

        print(f"[{self.name}] Starting worker on port {self.port}...")
        self.process = subprocess.Popen(
            cmd,
            stdout=subprocess.DEVNULL,
            stderr=subprocess.PIPE,
            start_new_session=True,
            env=env
        )
        self._wait_for_ready()

    def _wait_for_ready(
        self,
        timeout: float = 30.0 # Max seconds to wait for worker startup
    ) -> None:
        """Wait for worker to become responsive."""
        start = time.time()
        while time.time() - start < timeout:
            try:
                with httpx.Client() as client:
                    client.get(f"{self.base_url}/health")
                print(f"[{self.name}] Worker ready.")
                return
            except httpx.ConnectError:
                time.sleep(0.5)
        
        # Timeout - get stderr for debugging
        _, stderr = self.process.communicate(timeout=1)
        raise TimeoutError(
            f"Plugin '{self.name}' failed to start within {timeout}s: "
            f"{stderr.decode() if stderr else 'Unknown error'}"
        )


    def initialize(
        self,
        config: Optional[Dict[str, Any]] = None # Configuration dictionary
    ) -> None:
        """Initialize or reconfigure the plugin."""
        with httpx.Client() as client:
            resp = client.post(f"{self.base_url}/initialize", json=config or {})
        if resp.status_code != 200:
            raise RuntimeError(f"Initialize failed: {resp.text}")
    
    def execute(
        self,
        *args,
        **kwargs
    ) -> Any: # Plugin result
        """Execute the plugin synchronously."""
        payload = self._prepare_payload(args, kwargs)
        # timeout=None for long-running AI tasks
        with httpx.Client(timeout=None) as client:
            resp = client.post(f"{self.base_url}/execute", json=payload)
        
        if resp.status_code != 200:
            raise RuntimeError(f"Execute failed: {resp.text}")
        return resp.json()
    
    def get_config_schema(self) -> Dict[str, Any]: # JSON Schema
        """Get the plugin's configuration schema."""
        with httpx.Client() as client:
            return client.get(f"{self.base_url}/config_schema").json()
    
    def get_current_config(self) -> Dict[str, Any]: # Current config values
        """Get the plugin's current configuration."""
        with httpx.Client() as client:
            return client.get(f"{self.base_url}/config").json()


    def cleanup(self) -> None:
        """Clean up plugin resources and terminate worker process."""
        # Send cleanup request to worker
        try:
            with httpx.Client(timeout=2) as client:
                client.post(f"{self.base_url}/cleanup")
        except Exception:
            pass  # Worker may already be gone
        
        # Terminate the subprocess
        if self.process:
            self.process.terminate()
            try:
                self.process.wait(timeout=2)
            except subprocess.TimeoutExpired:
                self.process.kill()
            self.process = None

### Input Serialization

The proxy detects `FileBackedDTO` objects and serializes them to temp files before transmission. This enables zero-copy transfer of large data (audio, images) between processes.

In [ ]:
#| export
def _maybe_serialize_input(
    self,
    obj: Any # Object to potentially serialize
) -> Any: # Serialized form (path string or original object)
    """Convert FileBackedDTO objects to file paths for zero-copy transfer."""
    # Zero-Copy: Ask object to save itself
    if isinstance(obj, FileBackedDTO):
        return obj.to_temp_file()
    # Paths: Convert to string
    if isinstance(obj, Path):
        return str(obj)
    # Default: Pass through for JSON serialization
    return obj

def _prepare_payload(
    self,
    args: tuple, # Positional arguments
    kwargs: dict # Keyword arguments
) -> Dict[str, Any]: # JSON-serializable payload
    """Prepare arguments for HTTP transmission."""
    safe_args = [self._maybe_serialize_input(a) for a in args]
    safe_kwargs = {k: self._maybe_serialize_input(v) for k, v in kwargs.items()}
    return {"args": safe_args, "kwargs": safe_kwargs}

RemotePluginProxy._maybe_serialize_input = _maybe_serialize_input
RemotePluginProxy._prepare_payload = _prepare_payload

### Asynchronous Interface

These methods are `async` for use with FastHTML and other async frameworks. Use `execute_stream` for real-time streaming results.

In [ ]:
#| export
async def execute_async(
    self,
    *args,
    **kwargs
) -> Any: # Plugin result
    """Execute the plugin asynchronously."""
    payload = self._prepare_payload(args, kwargs)
    async with httpx.AsyncClient(timeout=None) as client:
        resp = await client.post(f"{self.base_url}/execute", json=payload)
    
    if resp.status_code != 200:
        raise RuntimeError(f"Execute failed: {resp.text}")
    return resp.json()

def execute_stream_sync(self, *args, **kwargs) -> Generator[Any, None, None]:
    """Synchronous wrapper for streaming (blocking)."""
    # This is tricky without "httpx.stream" in sync mode.
    # For now, it's okay to leave it async-only if documented.
    pass

async def execute_stream(
    self,
    *args,
    **kwargs
) -> AsyncGenerator[Any, None]: # Yields parsed JSON chunks
    """Execute with streaming response (async generator)."""
    payload = self._prepare_payload(args, kwargs)
    async with httpx.AsyncClient(timeout=None) as client:
        async with client.stream("POST", f"{self.base_url}/execute_stream", json=payload) as resp:
            async for line in resp.aiter_lines():
                if line:
                    yield json.loads(line)

RemotePluginProxy.execute_async = execute_async
RemotePluginProxy.execute_stream_sync = execute_stream_sync
RemotePluginProxy.execute_stream = execute_stream

### Lifecycle Management

In [ ]:
    #| export
def get_stats(self) -> Dict[str, Any]: # Process telemetry
    """Get worker process resource usage."""
    with httpx.Client() as client:
        return client.get(f"{self.base_url}/stats").json()

def is_alive(self) -> bool: # True if worker is responsive
    """Check if the worker process is still running and responsive."""
    if not self.process or self.process.poll() is not None:
        return False
    try:
        with httpx.Client(timeout=2) as client:
            resp = client.get(f"{self.base_url}/health")
            return resp.status_code == 200
    except Exception:
        return False

RemotePluginProxy.get_stats = get_stats
RemotePluginProxy.is_alive = is_alive

### Context Manager Support

The proxy can be used as a context manager for automatic cleanup.

In [ ]:
#| export
def __enter__(self):
    """Enter context manager."""
    return self

def __exit__(self, exc_type, exc_val, exc_tb):
    """Exit context manager and cleanup."""
    self.cleanup()
    return False

async def __aenter__(self):
    """Enter async context manager."""
    return self

async def __aexit__(self, exc_type, exc_val, exc_tb):
    """Exit async context manager and cleanup."""
    self.cleanup()
    return False

RemotePluginProxy.__enter__ = __enter__
RemotePluginProxy.__exit__ = __exit__
RemotePluginProxy.__aenter__ = __aenter__
RemotePluginProxy.__aexit__ = __aexit__

## Usage Examples

### Basic Usage (Sync)

```python
# Load manifest from JSON file
with open("~/.cjm/plugins/whisper.json") as f:
    manifest = json.load(f)

# Create proxy (starts worker subprocess)
plugin = RemotePluginProxy(manifest)

# Use like a local plugin
plugin.initialize({"model": "large-v3"})
result = plugin.execute(audio="/path/to/audio.wav")

# Clean up
plugin.cleanup()
```

### With Context Manager

```python
with RemotePluginProxy(manifest) as plugin:
    plugin.initialize({"model": "large-v3"})
    result = plugin.execute(audio="/path/to/audio.wav")
# Worker automatically terminated
```

### Async with Streaming (FastHTML)

```python
async def transcribe_stream(audio_path: str):
    async with RemotePluginProxy(manifest) as plugin:
        await plugin.initialize({"model": "large-v3"})
        async for chunk in plugin.execute_stream(audio=audio_path):
            yield chunk  # Stream to client
```

## Manifest Format

The proxy expects a manifest dictionary with at minimum:

```json
{
    "name": "whisper-local",
    "version": "1.0.0",
    "python_path": "/home/user/anaconda3/envs/cjm-whisper/bin/python",
    "module": "cjm_transcription_plugin_whisper.plugin",
    "class": "WhisperLocalPlugin",
    "env_vars": {
        "CUDA_VISIBLE_DEVICES": "0"
    }
}
```

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()